In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.7 MB/s eta 0:00:00


In [ ]:
#Mount drive to store files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import datetime

In [ ]:
# Load the Original Dataset
from datasets import load_dataset
dataset = load_dataset("iamtarun/python_code_instructions_18k_alpaca")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/18612 [00:00<?, ? examples/s]

In [ ]:
# Explore Data
datapoint = dataset['train'][0:2]
datapoint['instruction'], datapoint['input'], datapoint['output']

(['Create a function to calculate the sum of a sequence of integers.',
  'Generate a Python code for crawling a website for a specific type of data.'],
 ['[1, 2, 3, 4, 5]',
  'website: www.example.com \ndata to crawl: phone numbers'],
 ['# Python code\ndef sum_sequence(sequence):\n  sum = 0\n  for num in sequence:\n    sum += num\n  return sum',
  "import requests\nimport re\n\ndef crawl_website_for_phone_numbers(website):\n    response = requests.get(website)\n    phone_numbers = re.findall('\\d{3}-\\d{3}-\\d{4}', response.text)\n    return phone_numbers\n    \nif __name__ == '__main__':\n    print(crawl_website_for_phone_numbers('www.example.com'))"])

In [ ]:
#Create a function with prompt to augment data with gpt3.5
from openai import OpenAI

def get_chat_output(instruction, input, output):
  client = OpenAI(
      # defaults to os.environ.get("OPENAI_API_KEY")
      api_key="KEY",
  )

  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "system",
              "content": "below is an instruction for a programming problem, with an input(if availlable) and an output of generated Python code. I want you to add short, concise, comments into the code relating to the problem statement, concept, or flow of control that is required to understand the code and no comments about variable assignment, arithmetic, or I/O. It should not affect the original code and return this new code only."+
                        f" Instruction: {instruction}" +
                        f" Input: {input}" +
                        f" Code: {output}",
          }
      ],
      model="gpt-3.5-turbo",
  )
  return chat_completion

In [ ]:
#Keep track of tokens used
tokens_used = 0

In [ ]:
import threading
import time
#Function to get chat completion from the openai function and extract the outputs and tokens
def process_data(start, end, outputs, tokens_used):
    start_time = datetime.datetime.now()
    for i in range(start, end):
        datapoint = dataset['train'][i]
        chat_completion = get_chat_output(instruction=datapoint['instruction'], input=datapoint['input'], output=datapoint['output'])
        output = {'i': i, 'new_code': chat_completion.choices[0].message.content, 'instruction': datapoint['instruction'], 'input': datapoint['input'], 'output': datapoint['output']}
        tokens_used += chat_completion.usage.total_tokens
        print(f"\n######## i:{i}, tokens_used:{tokens_used}, output:{output}")
        time_elapsed = datetime.datetime.now() - start_time
        print(f"Time elapsed: {time_elapsed.total_seconds()} s")
        outputs.append(output)

#create a multithread process to send requests in parallel as one request takes about 8-9 seconds to complete
outputs = []
if __name__ == "__main__":

    threads = []

    # Divide the loop into multiple ranges based on the desired number of threads
    num_threads = 4  # Adjust this value to the desired number of threads
    start = 10000
    end = 11000
    for i in range(start, end, num_threads):
        thread = threading.Thread(target=process_data, args=(i, i + num_threads, outputs, tokens_used))
        threads.append(thread)
        thread.start()

    # Wait for all threads to finish
    for thread in threads:
        thread.join()
    json_data = json.dumps(outputs)
    #write data to a json file in drive
    with open(f'/content/drive/MyDrive/alpaca_dataset/alpaca_comment_shard_{start}_{end}.json', 'w') as json_file:
        json_file.write(json_data)



######## i:10508, tokens_used:191, output:{'i': 10508, 'new_code': 'def reverse_list(L2):\n    L2.reverse()  # Reverses the order of the elements in L2\n    return L2  # Returns the reversed list L2', 'instruction': 'Once you have generated a list of strings, write a Python program to print the same in reverse order.', 'input': 'L2= ["This", "is", "another", "list", "of", "strings"]', 'output': 'def reverse_list(L2):\n    L2.reverse()\n    return L2'}
Time elapsed: 22.980077 s

######## i:10676, tokens_used:181, output:{'i': 10676, 'new_code': 'import random\n\n# Generate N random numbers \n# using list comprehension and random.randint\n# Store the numbers in a list called "numbers"\nnumbers = [random.randint(0, N) for i in range(N)]', 'instruction': 'Generate a syntax to input random numbers into a python program', 'input': 'N = 10', 'output': 'import random\n\n# Generates N random numbers\nnumbers = [random.randint(0, N) for i in range(N)]'}
Time elapsed: 23.796453 s

######## i:104

Exception in thread Thread-160 (process_data):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-bad741d0e7a6>", line 8, in process_data
  File "<ipython-input-6-f972c120a848>", line 9, in get_chat_output
  File "/usr/local/lib/python3.10/dist-packages/openai/_utils/_utils.py", line 299, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/openai/resources/chat/completions.py", line 598, in create
    return self._post(
  File "/usr/local/lib/python3.10/dist-packages/openai/_base_client.py", line 1063, in post
    return cast(ResponseT, self.request(cast_to, opts, stream=stream, stream_cls=stream_cls))
  File "/usr/local/lib/python3.10/dist-packages/openai/_base_client.py", line 842, in request
    return self._request(
  File "/usr/local/lib/pyt


######## i:10853, tokens_used:610, output:{'i': 10853, 'new_code': 'def remove_duplicates(list):\n    new_list = []  # create an empty list to hold the unique elements\n    for e in list:  # iterate through each element in the given list\n        if e not in new_list:  # check if the element is not already in the new list\n            new_list.append(e)  # if not, add it to the new list\n    return new_list\n\nlist = [1, 2, 3, 1, 3, 4]  # initialize the given list\nlist = remove_duplicates(list)  # call the function to remove duplicates and store the result in the same variable\nprint(list)  # print the updated list', 'instruction': 'Create a program in Python to delete all duplicates from a given list.', 'input': 'list = [1, 2, 3, 1, 3, 4]', 'output': 'def remove_duplicates(list):\n    new_list = []\n    for e in list:\n        if e not in new_list:\n            new_list.append(e)\n    return new_list\n\nlist = [1, 2, 3, 1, 3, 4]\nlist = remove_duplicates(list)\nprint(list)'}
Time el

Exception in thread Thread-57 (process_data):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-bad741d0e7a6>", line 8, in process_data
  File "<ipython-input-6-f972c120a848>", line 9, in get_chat_output
  File "/usr/local/lib/python3.10/dist-packages/openai/_utils/_utils.py", line 299, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/openai/resources/chat/completions.py", line 598, in create
    return self._post(
  File "/usr/local/lib/python3.10/dist-packages/openai/_base_client.py", line 1063, in post
    return cast(ResponseT, self.request(cast_to, opts, stream=stream, stream_cls=stream_cls))
  File "/usr/local/lib/python3.10/dist-packages/openai/_base_client.py", line 842, in request
    return self._request(
  File "/usr/local/lib/pyth


######## i:10553, tokens_used:1762, output:{'i': 10553, 'new_code': 'import random\n\n# iterate 10 times\nfor i in range(10):\n  # generate a random number between 1 and 15\n  print(random.randint(1, 15))', 'instruction': 'Write a Python script to generate 10 random numbers between 1 and 15.', 'input': '', 'output': 'import random\n\nfor i in range(10):\n  print(random.randint(1, 15))'}
Time elapsed: 129.875195 s

######## i:10479, tokens_used:1610, output:{'i': 10479, 'new_code': 'def fibonacci(n): \n    if n <= 1: # if n is 0 or 1\n        return n \n    else:\n        fibonacci_num=[0,1] #creating a fibonacci list with first two numbers\n        i=2 # start at index 2\n        while i<=n: # loop until index i is greater than n\n            fibonacci_num.append(fibonacci_num[i-1]+fibonacci_num[i-2]) # add the sum of the previous two numbers to the list\n            i+=1 # increment index i by 1\n        return fibonacci_num[n] # return the nth fibonacci number from the list of fibon

...


######## i:10642, tokens_used:2903, output:{'i': 10642, 'new_code': '#----------------------------------------------------------------------------------------\n# Copyright, 2013:\n#\n# Stefano Ermon \t\t- Cornell University \t\t\t, ermonste@cs.cornell.edu\n# Ashish Sabharwal \t\t- IBM Watson Research Center \t, ashish.sabharwal@us.ibm.com\n#----------------------------------------------------------------------------------------\n\nimport sys\nimport math\nimport random\nimport os\nimport argparse\nfrom WISHLogProcess import process_logs \nfrom WISHLogProcess import process_logs_cplex_LB\nfrom WISHLogProcess import process_logs_cplex_UB\n\n# version number\n__version__ = \'1.0\'\n\n#########################################\n# Usage Information:\n# run "python WISH.py -h" for help\n#########################################\n\nparser = argparse.ArgumentParser(description=\'Estimate the partition function using the WISH algorithm and CPLEX for the optimization.\')\n\nparser.add_argument(\

Exception in thread Thread-78 (process_data):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-bad741d0e7a6>", line 8, in process_data
  File "<ipython-input-6-f972c120a848>", line 9, in get_chat_output
  File "/usr/local/lib/python3.10/dist-packages/openai/_utils/_utils.py", line 299, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/openai/resources/chat/completions.py", line 598, in create
    return self._post(
  File "/usr/local/lib/python3.10/dist-packages/openai/_base_client.py", line 1063, in post
    return cast(ResponseT, self.request(cast_to, opts, stream=stream, stream_cls=stream_cls))
  File "/usr/local/lib/python3.10/dist-packages/openai/_base_client.py", line 842, in request
    return self._request(
  File "/usr/local/lib/pyth


######## i:10426, tokens_used:1451, output:{'i': 10426, 'new_code': 'def max_list_div_conq(lst): \n    if len(lst) == 1:  # Check if the list contains only one element\n        return lst[0]   # If so, return that element\n  \n    mid = len(lst)//2   # Find the middle index of the list\n    left_max = max_list_div_conq(lst[:mid])  # Recursively find the maximum element in the left half of the list \n    right_max = max_list_div_conq(lst[mid:]) # Recursively find the maximum element in the right half of the list \n    return max(left_max, right_max)  # Return the maximum element between the left and right halves', 'instruction': 'Construct a python program to find the maximum element in a list using divide and conquer approach.', 'input': '', 'output': 'def max_list_div_conq(lst): \n    if len(lst) == 1: \n        return lst[0] \n  \n    mid = len(lst)//2\n    left_max = max_list_div_conq(lst[:mid]) \n    right_max = max_list_div_conq(lst[mid:]) \n    return max(left_max, right_max)'}


Exception in thread Thread-193 (process_data):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-bad741d0e7a6>", line 8, in process_data
  File "<ipython-input-6-f972c120a848>", line 9, in get_chat_output
  File "/usr/local/lib/python3.10/dist-packages/openai/_utils/_utils.py", line 299, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/openai/resources/chat/completions.py", line 598, in create
    return self._post(
  File "/usr/local/lib/python3.10/dist-packages/openai/_base_client.py", line 1063, in post
    return cast(ResponseT, self.request(cast_to, opts, stream=stream, stream_cls=stream_cls))
  File "/usr/local/lib/python3.10/dist-packages/openai/_base_client.py", line 842, in request
    return self._request(
  File "/usr/local/lib/pyt


######## i:10599, tokens_used:1223, output:{'i': 10599, 'new_code': 'from sklearn.datasets import load_iris\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.tree import DecisionTreeClassifier\nfrom sklearn.metrics import accuracy_score\n\n# Load the dataset\niris = load_iris()\n\n# Split data into training and test sets\nX_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=0)\n\n# Fit the decision tree model to the training data\nclf = DecisionTreeClassifier(random_state=0).fit(X_train, y_train) \n \n# Make predictions on the test data\ny_pred = clf.predict(X_test) # Predict the color of the flowers in the test data using the decision tree model\n\n# Calculate accuracy \nacc = accuracy_score(y_test, y_pred) # Compare the predicted colors with the actual colors and calculate the accuracy\nprint(acc) # Print the accuracy of the model', 'instruction': 'Develop a classification model in Python to predict the color of a 

In [ ]:
#How many requests went through and tokens used
len(outputs), tokens_used

In [ ]:
#Find out which id requests were aborted due to rate limit to use them later
lst = [i['i'] for i in outputs]
lst.sort()
for i in range(2000,3000):
  if i not in lst:
    print(i)

2826
2827
2885
2886
2887


In [ ]:
#merge all different json files into one big dataset
import os, json
files = os.listdir("/content/drive/MyDrive/alpaca_dataset")
print(files)
def merge_JsonFiles(filename):
    result = list()
    for f1 in filename:
      if f1.endswith('.json'):
        f1 = "/content/drive/MyDrive/alpaca_dataset/" + f1
        with open(f1, 'r') as infile:
            bef = len(result)
            result.extend(json.load(infile))
            print(f1, len(result)-bef)
    with open('code_alpaca_python_comments.json', 'w') as output_file:
        print(len(result))
        json.dump(result, output_file)

merge_JsonFiles(files)

['alpaca_comment_shard_0_10.json', 'alpaca_comment_shard_10_500.json', '.ipynb_checkpoints', 'alpaca_comment_shard_500_1000.json', 'alpaca_comment_shard_1000_1500.json', 'alpaca_comment_shard_1500_2000.json', 'alpaca_comment_shard_2000_3000.json', 'alpaca_comment_shard_3000_4000.json', 'alpaca_comment_shard_4000_5000.json', 'alpaca_comment_shard_5000_7000.json', 'alpaca_comment_shard_7000_8000.json', 'alpaca_comment_shard_8000_10000.json', 'alpaca_comment_shard_10000_11000.json']
/content/drive/MyDrive/alpaca_dataset/alpaca_comment_shard_0_10.json 10
/content/drive/MyDrive/alpaca_dataset/alpaca_comment_shard_10_500.json 10
/content/drive/MyDrive/alpaca_dataset/alpaca_comment_shard_500_1000.json 500
/content/drive/MyDrive/alpaca_dataset/alpaca_comment_shard_1000_1500.json 500
/content/drive/MyDrive/alpaca_dataset/alpaca_comment_shard_1500_2000.json 500
/content/drive/MyDrive/alpaca_dataset/alpaca_comment_shard_2000_3000.json 995
/content/drive/MyDrive/alpaca_dataset/alpaca_comment_shard